In [ ]:
import ee
import time
from datetime import datetime
from utils import check_task_status

ee.Authenticate()
ee.Initialize(project='dse-staff')

GSW = ee.Image("JRC/GSW1_2/GlobalSurfaceWater").select("max_extent")
HM_IMAGE = ee.ImageCollection('CSP/HM/GlobalHumanModification').mean()
MODIS = ee.ImageCollection('MODIS/006/MOD09A1')

samples = ee.FeatureCollection('projects/dse-staff/assets/transects_final')

# Generate folder name with current date
folder_name = f"boundary-project_results_{datetime.now().strftime('%Y%m%d')}"
print(f"Results will be exported to Google Drive folder: {folder_name}")

/usr/local/lib/python3.10/dist-packages/ee/deprecation.py:209: DeprecationWarning: 

Attention required for JRC/GSW1_2/GlobalSurfaceWater! You are using a deprecated asset.
To make sure your code keeps working, please update it.
Learn more: https://developers.google.com/earth-engine/datasets/catalog/JRC_GSW1_2_GlobalSurfaceWater

  warnings.warn(warning, category=DeprecationWarning)
/usr/local/lib/python3.10/dist-packages/ee/deprecation.py:209: DeprecationWarning: 

Attention required for MODIS/006/MOD09A1! You are using a deprecated asset.
To make sure your code keeps working, please update it.
Learn more: https://developers.google.com/earth-engine/datasets/catalog/MODIS_006_MOD09A1

  warnings.warn(warning, category=DeprecationWarning)


In [2]:
water_results = GSW.reduceRegions(
    collection=samples,
    reducer=ee.Reducer.max().setOutputs(['water_max_extent']),
    scale=30,  # used 30m bc 500m misses many water pts
    tileScale=8
)

hm_results = HM_IMAGE.reduceRegions(
    collection=water_results,
    reducer=ee.Reducer.mean().combine(ee.Reducer.stdDev(), '', True).combine(ee.Reducer.median(), '', True)
        .setOutputs(['hm_mean', 'hm_std', 'hm_med']),
    scale=500,
    tileScale=8
)

In [ ]:
# Process multiple years of MODIS gradient data with task queue management
years = list(range(2001, 2002)) 
max_concurrent_tasks = 15
submitted_tasks = []

for i, year in enumerate(years):
    # Wait if we have too many active tasks
    submitted_tasks, n_active = check_task_status(submitted_tasks)
    while n_active >= max_concurrent_tasks:
        print(f"Waiting... {n_active} tasks active")
        time.sleep(30)  # Check every 30 seconds
        submitted_tasks, n_active = check_task_status(submitted_tasks)
    
    print(f"Processing year {year} ({i+1}/{len(years)})...")
    
    # Get max MODIS image and calculate NDVI
    modis = MODIS.filterDate(f'{year}-01-01', f'{year}-12-31') \
        .max() \
        .select(['sur_refl_b01', 'sur_refl_b02'])  
    
    # Calculate NDVI
    ndvi = modis.normalizedDifference(['sur_refl_b02', 'sur_refl_b01']).rename('ndvi').select('ndvi')
    
    # Calculate gradient 
    grad = ndvi.gradient()
    magnitude = grad.expression('sqrt(x*x + y*y)', {'x': grad.select('x'), 'y': grad.select('y')})

    # Reduce with explicit CRS and scale matching MODIS
    final_results = magnitude.reduceRegions(
        collection=hm_results,
        reducer=ee.Reducer.mean().combine(ee.Reducer.stdDev(), '', True).combine(ee.Reducer.count(), '', True)
            .setOutputs(['grad_mean', 'grad_std', 'grad_count']),
        scale=500,   
        tileScale=8
    )
    
    # Export results and track task
    export_task = ee.batch.Export.table.toDrive(
        collection=final_results,
        description=f'ndvi_results_{year}',
        folder=folder_name,
        fileNamePrefix=f'ndvi_results_{year}',
        fileFormat='CSV',
        selectors=['WDPA_PID', 'transectID', 'point_position', 
        'water_max_extent', 'hm_mean', 'hm_std', 'hm_med', 
        'grad_mean', 'grad_std', 'grad_count']
    )
    export_task.start()
    submitted_tasks.append((export_task, year))
    print(f"Task submitted for {year}")

# Wait for all remaining tasks to complete
print("\nWaiting for all tasks to complete...")
submitted_tasks, n_active = check_task_status(submitted_tasks)
while n_active > 0:
    print(f"{n_active} tasks still running...")
    time.sleep(30)
    submitted_tasks, n_active = check_task_status(submitted_tasks)

print("All tasks completed!")

Processing year 2001 (1/1)...
Task submitted for 2001

Waiting for all tasks to complete...
1 tasks still running...
1 tasks still running...
1 tasks still running...
1 tasks still running...
1 tasks still running...
1 tasks still running...
1 tasks still running...
1 tasks still running...
1 tasks still running...
1 tasks still running...
1 tasks still running...
1 tasks still running...
1 tasks still running...
1 tasks still running...
1 tasks still running...
1 tasks still running...
1 tasks still running...
1 tasks still running...
1 tasks still running...
1 tasks still running...
1 tasks still running...
1 tasks still running...
1 tasks still running...
1 tasks still running...
1 tasks still running...
1 tasks still running...
1 tasks still running...
1 tasks still running...
1 tasks still running...
1 tasks still running...
1 tasks still running...
1 tasks still running...
1 tasks still running...
1 tasks still running...
1 tasks still running...
1 tasks still running...
1 tasks 